In [1]:
import os
import re
import pandas as pd

main_dir = '/home/vault/empkins/tpD/D02/RCT/raw_data'

patient_ids = []
mp4_paths = []
file_existence = []

filename_pattern = re.compile(r'(\w+)_t2_')

for root, dirs, files in os.walk(main_dir):
    if root == main_dir:  
        for patient_id in dirs:
            patient_folder = os.path.join(root, patient_id)

            mp4_files = [file for file in os.listdir(patient_folder) if file.endswith('.mp4')]

            for mp4_file in mp4_files:
                match = filename_pattern.match(mp4_file)
                if match:
                    patient_ids.append(patient_id)
                    mp4_paths.append(os.path.join(patient_folder, mp4_file))
                    file_existence.append(True)
                    break  

            if not any(match for match in filename_pattern.findall(' '.join(mp4_files))):
                patient_ids.append(patient_id)
                mp4_paths.append(None)
                file_existence.append(False)

data = {
    'Patient_ID': patient_ids,
    'MP4_Path': mp4_paths,
    'File_Exists': file_existence
}

df = pd.DataFrame(data)

print(df)

    Patient_ID                                           MP4_Path  File_Exists
0          338  /home/vault/empkins/tpD/D02/RCT/raw_data/338/3...         True
1          263  /home/vault/empkins/tpD/D02/RCT/raw_data/263/2...         True
2          251  /home/vault/empkins/tpD/D02/RCT/raw_data/251/2...         True
3          439  /home/vault/empkins/tpD/D02/RCT/raw_data/439/4...         True
4          310  /home/vault/empkins/tpD/D02/RCT/raw_data/310/3...         True
..         ...                                                ...          ...
150        287  /home/vault/empkins/tpD/D02/RCT/raw_data/287/2...         True
151        120  /home/vault/empkins/tpD/D02/RCT/raw_data/120/1...         True
152        308  /home/vault/empkins/tpD/D02/RCT/raw_data/308/3...         True
153        162  /home/vault/empkins/tpD/D02/RCT/raw_data/162/1...         True
154        221  /home/vault/empkins/tpD/D02/RCT/raw_data/221/2...         True

[155 rows x 3 columns]


In [2]:
df.to_csv('/home/vault/empkins/tpD/D02/processed_data/sonycam_videos_dir2.csv')

In [44]:
import subprocess

target_folder = '/home/vault/empkins/tpD/D02/processed_data/SonyCam_processed_data'


def execute_command(command, patient_id):
    try:
        subprocess.run(command, shell=True, check=True)
        print(f"Command executed successfully for patient ID: {patient_id}")
    except subprocess.CalledProcessError as e:
        print(f"Command execution failed for patient ID: {patient_id}. Error: {e}")

for index, row in df[df['File_Exists']].iterrows():
    patient_id = row['Patient_ID']
    mp4_path = row['MP4_Path']
    
    input_path = mp4_path  
    output_folder = os.path.join(target_folder, patient_id)
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder)  
    
    command_1 = "export https_proxy=http://proxy:80"
    execute_command(command_1, patient_id)

    command_2 = "export http_proxy=http://proxy:80"
    execute_command(command_2, patient_id)
    
    command_3 = f'apptainer exec --writable-tmpfs /home/vault/empkins/tpD/D02/processed_data/SonyCam_processed_data/dbm-openface-2.sif /bin/bash -c "cd /app; python3 -W ignore /app/process_data.py --input_path {input_path} --output_path {output_path}"'
    execute_command(command_3, patient_id)

Command executed successfully for patient ID: 251
Command executed successfully for patient ID: 251


Traceback (most recent call last):
  File "/app/process_data.py", line 17, in <module>
    from opendbm.dbm_lib import config_derive_feature, config_raw_feature, config_reader
  File "/app/opendbm/__init__.py", line 1, in <module>
    from opendbm.api_lib.facial_activity import FacialActivity
  File "/app/opendbm/api_lib/__init__.py", line 1, in <module>
    from .model import DEEEPSPEECH_URL, DEEPSPEECH_MODELS, OPENDBM_DATA, AudioModel
  File "/app/opendbm/api_lib/model.py", line 7, in <module>
    from opendbm.api_lib.util import docker_command_dec, wsllize
  File "/app/opendbm/api_lib/util.py", line 11, in <module>
    from opendbm.dbm_lib.controller import process_feature as pf
  File "/app/opendbm/dbm_lib/__init__.py", line 16, in <module>
    from .dbm_features.raw_features.nlp.speech_features import run_speech_feature
  File "/app/opendbm/dbm_lib/dbm_features/raw_features/nlp/__init__.py", line 1, in <module>
    from .speech_features import run_speech_feature
  File "/app/opend

Command execution failed for patient ID: 251. Error: Command 'apptainer exec --writable-tmpfs /home/vault/empkins/tpD/D02/processed_data/SonyCam_processed_data/dbm-openface-2.sif /bin/bash -c "cd /app; python3 -W ignore /app/process_data.py --input_path /home/vault/empkins/tpD/D02/RCT/raw_data/251/251_t2_20230911.mp4 --output_path /home/vault/empkins/tpD/D02/processed_data/SonyCam_processed_data/251"' returned non-zero exit status 1.


In [43]:
os.listdir('/scratch/')

['.privtmp']

In [ ]:
/home/vault/empkins/tpD/D02/RCT/open_face_processed_data/147

In [4]:
import os

def list_missing_folders(dir1, dir2):
    # Get list of folders in dir1
    folders1 = set(next(os.walk(dir1))[1])
    
    # Get list of folders in dir2
    folders2 = set(next(os.walk(dir2))[1])
    
    # Find folders in dir1 but not in dir2
    missing_folders = folders1 - folders2
    
    return missing_folders

# Provide paths to the two directories
dir1_path = "/home/vault/empkins/tpD/D02/RCT/raw_data"
dir2_path = "/home/vault/empkins/tpD/D02/processed_data/SonyCam_processed_data"

missing_folders = list_missing_folders(dir1_path, dir2_path)
print("Folders missing in dir2:")

Folders missing in dir2:


In [6]:
len(missing_folders)

41

In [10]:
missing_folders

{'008',
 '044',
 '093',
 '120',
 '131',
 '146',
 '151',
 '162',
 '221',
 '241',
 '263',
 '306',
 '316',
 '321',
 '325',
 '327',
 '337',
 '338',
 '385',
 '403',
 '418',
 '430',
 '438',
 '439',
 '444',
 '447',
 '453',
 '459',
 '460',
 '467',
 '469',
 '471',
 '472',
 '476',
 '482',
 '507',
 '512',
 '516',
 '518',
 '559',
 'Radar_DB'}